In [22]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from config import g_key

In [23]:
flights_df = pd.read_csv("Summary Flight Tables/airport_summary.csv",index_col=0)
flights_df

,Airport,# Cancelled Flights,# Non-Cancelled Flights,Total Flights
0,DFW,10181,900961,911142
1,ATL,7701,1505703,1513404
2,DEN,5989,891080,897069
3,LAX,2372,853024,855396
4,ORD,13418,1133618,1147036


In [24]:
flights_df["Latitude"]=""
flights_df["Longitude"]=""
flights_df

,Airport,# Cancelled Flights,# Non-Cancelled Flights,Total Flights,Latitude,Longitude
0,DFW,10181,900961,911142,,
1,ATL,7701,1505703,1513404,,
2,DEN,5989,891080,897069,,
3,LAX,2372,853024,855396,,
4,ORD,13418,1133618,1147036,,


In [25]:
coordinate_df = pd.read_csv("Monthly_Weather_Averages.csv")
coordinate_df

,Unnamed: 0,Date,Latitude,Longitude,Max Temp,Precip,Wind Speed,Airport
0,0,2015-01-31,32.896,-97.037,53.941935,0.116129,16.906452,DFW
1,1,2015-02-28,32.896,-97.037,55.828571,0.107143,19.442857,DFW
2,2,2015-03-31,32.896,-97.037,64.629032,0.082258,15.983871,DFW
3,3,2015-04-30,32.896,-97.037,74.166667,0.185000,18.783333,DFW
4,4,2015-05-31,32.896,-97.037,78.296774,0.548710,19.332258,DFW
...,...,...,...,...,...,...,...,...
235,43,2018-08-31,41.979,-87.904,83.903226,0.218387,15.654839,ORD
236,44,2018-09-30,41.979,-87.904,77.060000,0.121333,14.876667,ORD
237,45,2018-10-31,41.979,-87.904,60.409677,0.154839,15.445161,ORD
238,46,2018-11-30,41.979,-87.904,39.430000,0.074667,16.456667,ORD


In [26]:
flights_list = flights_df["Airport"].tolist()
flights_list

['DFW', 'ATL', 'DEN', 'LAX', 'ORD']

In [27]:
dfw_lat = coordinate_df.loc[coordinate_df["Airport"]=="DFW"]["Latitude"].unique()
dfw_lng = coordinate_df.loc[coordinate_df["Airport"]=="DFW"]["Longitude"].unique()
atl_lat = coordinate_df.loc[coordinate_df["Airport"]=="ATL"]["Latitude"].unique()
atl_lng = coordinate_df.loc[coordinate_df["Airport"]=="ATL"]["Longitude"].unique()
den_lat = coordinate_df.loc[coordinate_df["Airport"]=="DEN"]["Latitude"].unique()
den_lng = coordinate_df.loc[coordinate_df["Airport"]=="DEN"]["Longitude"].unique()
lax_lat = coordinate_df.loc[coordinate_df["Airport"]=="LAX"]["Latitude"].unique()
lax_lng = coordinate_df.loc[coordinate_df["Airport"]=="LAX"]["Longitude"].unique()
ord_lat = coordinate_df.loc[coordinate_df["Airport"]=="ORD"]["Latitude"].unique()
ord_lng = coordinate_df.loc[coordinate_df["Airport"]=="ORD"]["Longitude"].unique()   
lat_list = [dfw_lat[0],atl_lat[0],den_lat[0],lax_lat[0],ord_lat[0]]
lng_list = [dfw_lng[0],atl_lng[0],den_lng[0],lax_lng[0],ord_lng[0]]
dict = {"Airport": flights_list,
        "Latitude":lat_list,
        "Longitude":lng_list}
df=pd.DataFrame(dict)
df

,Airport,Latitude,Longitude
0,DFW,32.896,-97.037
1,ATL,33.640,-84.427
2,DEN,39.858,-104.667
3,LAX,33.942,-118.408
4,ORD,41.979,-87.904


In [28]:
final_df=pd.merge(flights_df,df, on="Airport")
final_df.rename(columns={"Latitude_y":"Latitude","Longitude_y":"Longitude"},inplace = True)
final_df=final_df[["Airport","# Cancelled Flights","# Non-Cancelled Flights","Total Flights","Latitude","Longitude"]]
final_df

,Airport,# Cancelled Flights,# Non-Cancelled Flights,Total Flights,Latitude,Longitude
0,DFW,10181,900961,911142,32.896,-97.037
1,ATL,7701,1505703,1513404,33.640,-84.427
2,DEN,5989,891080,897069,39.858,-104.667
3,LAX,2372,853024,855396,33.942,-118.408
4,ORD,13418,1133618,1147036,41.979,-87.904


In [29]:
#configure gmaps with API key
gmaps.configure(api_key=g_key)

In [30]:
locations =final_df[["Latitude", "Longitude"]].astype(float)
weights = final_df["# Cancelled Flights"]

In [31]:
# Create HeatMap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations,weights = weights,
                              dissipating=False, max_intensity=100,
                              point_radius = 0.5)
# heat.gradient = ['yellow','blue']
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [32]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "keyword": "airport",
    "key": g_key}
final_df["Airport Name"]=""
#looping through the dataframe
for index,row in df.iterrows():
    lat=row["Latitude"]
    lng=row["Longitude"]
    params["location"]=f"{lat},{lng}"
    
     # Use the search term: "airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # make request and print url
    name = requests.get(base_url, params=params)
     # convert to json
    response = name.json()
    try:
        final_df.loc[index,"Airport Name"]=response['results'][0]['name']        
    except (KeyError, IndexError):
        print("name not found... skipping.")

In [33]:
final_df

,Airport,# Cancelled Flights,# Non-Cancelled Flights,Total Flights,Latitude,Longitude,Airport Name
0,DFW,10181,900961,911142,32.896,-97.037,Dallas/Fort Worth International Airport
1,ATL,7701,1505703,1513404,33.640,-84.427,Hartsfield-Jackson Atlanta International Airport
2,DEN,5989,891080,897069,39.858,-104.667,Denver International Airport
3,LAX,2372,853024,855396,33.942,-118.408,Los Angeles International Airport
4,ORD,13418,1133618,1147036,41.979,-87.904,O'Hare International Airport


In [36]:
# Using the template add the airport marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Airport Name}</dd>
<dt>Cancelled_Flights</dt><dd>{# Cancelled Flights}</dd>
<dt>Total_Flights</dt><dd>{Total Flights}</dd>
</dl>
"""
airport_info = [info_box_template.format(**row) for index, row in final_df.iterrows()]
locations = final_df[["Latitude", "Longitude"]].astype(float)

# Create HeatMap for the cancelled flights
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations,info_box_content=airport_info)
fig.add_layer(marker_layer)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [37]:
locations =final_df[["Latitude", "Longitude"]].astype(float)
weights = final_df["# Non-Cancelled Flights"]

fig = gmaps.figure()
heat_layer1 = gmaps.heatmap_layer(locations,weights = weights,
                              dissipating=False, max_intensity=100,
                              point_radius = 2)
# heat.gradient = ['yellow','blue']
fig.add_layer(heat_layer1)

# Using the template add the airport marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Airport Name}</dd>
<dt>Cancelled_Flights</dt><dd>{# Cancelled Flights}</dd>
<dt>NonCancelled_Flights</dt><dd>{# Non-Cancelled Flights}</dd>
<dt>Total_Flights</dt><dd>{Total Flights}</dd>
</dl>
"""
airport_info = [info_box_template.format(**row) for index, row in final_df.iterrows()]
locations = final_df[["Latitude", "Longitude"]].astype(float)

# Create HeatMap showing cancelled and Non-cancelled flights
fig = gmaps.figure()
marker_layer1 = gmaps.marker_layer(locations,info_box_content=airport_info)
fig.add_layer(marker_layer1)
fig.add_layer(heat_layer1)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))